In [19]:
# import Required libraries
import tweepy as tw
import pandas as pd
import re
import nltk
import glob
import string
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [2]:
nltk.download('words')
words = set(nltk.corpus.words.words())

[nltk_data] Downloading package words to
[nltk_data]     C:\Users\Raj\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!


In [3]:
moviefile = 'u.item.txt'
movieinfo = pd.read_csv(moviefile, sep="|", header=None, index_col=False,
                  names=['itemId','title'], usecols=[0,1])
movieinfo

,itemId,title
0,1,Toy Story (1995)
1,2,GoldenEye (1995)
2,3,Four Rooms (1995)
3,4,Get Shorty (1995)
4,5,Copycat (1995)
...,...,...
1677,1678,Mat' i syn (1997)
1678,1679,B. Monkey (1998)
1679,1680,Sliding Doors (1998)
1680,1681,You So Crazy (1994)


In [4]:
movieinfo['year'] = movieinfo.title.str.extract('(\(\d\d\d\d\))',expand=False)
movieinfo['year'] = movieinfo.year.str.extract('(\d\d\d\d)',expand=False)

#Removing the years from the 'title' column
movieinfo['title'] = movieinfo.title.str.replace('(\(\d\d\d\d\))', '')
movieinfo

<ipython-input-4-9d92f91aa118>:5: FutureWarning: The default value of regex will change from True to False in a future version.
  movieinfo['title'] = movieinfo.title.str.replace('(\(\d\d\d\d\))', '')


,itemId,title,year
0,1,Toy Story,1995
1,2,GoldenEye,1995
2,3,Four Rooms,1995
3,4,Get Shorty,1995
4,5,Copycat,1995
...,...,...,...
1677,1678,Mat' i syn,1997
1678,1679,B. Monkey,1998
1679,1680,Sliding Doors,1998
1680,1681,You So Crazy,1994


In [5]:
movieinfo['tagword'] = movieinfo['title'].str.lower()
movieinfo['tagword'] = movieinfo.tagword.str.replace(' ', '')
movieinfo['tagword'] = movieinfo['tagword'].apply(lambda x: "{}{}".format('#', x))
movieinfo['tagword'] = movieinfo.tagword.str.replace('(\([A-Za-z0-9_]+\))', '')
movieinfo

<ipython-input-5-e05924e0ad62>:4: FutureWarning: The default value of regex will change from True to False in a future version.
  movieinfo['tagword'] = movieinfo.tagword.str.replace('(\([A-Za-z0-9_]+\))', '')


,itemId,title,year,tagword
0,1,Toy Story,1995,#toystory
1,2,GoldenEye,1995,#goldeneye
2,3,Four Rooms,1995,#fourrooms
3,4,Get Shorty,1995,#getshorty
4,5,Copycat,1995,#copycat
...,...,...,...,...
1677,1678,Mat' i syn,1997,#mat'isyn
1678,1679,B. Monkey,1998,#b.monkey
1679,1680,Sliding Doors,1998,#slidingdoors
1680,1681,You So Crazy,1994,#yousocrazy


In [6]:
movieinfo['title'][75]

"Carlito's Way "

In [7]:
ACCESS_TOKEN = "1284760789660741633-59TnA3KaoM1yClzczVdUkoA3lqzglR"
ACCESS_TOKEN_SECRET = "3UJICOI5a62rVtkjEmDhxCyvEp8o8QvtPLIz0s13n2BPb"
CONSUMER_API_KEY = "9QllVs1sWUbF07RFXLUcXjx1V"
CONSUMER_API_SECRET = "F7sjrqkvajBCnEmUHNHRTxsNphqM6QDAo32YHP6eCQOcE8g0Jl"

In [8]:
# Create an Authentication Object
auth = tw.OAuthHandler(CONSUMER_API_KEY, CONSUMER_API_SECRET)

In [9]:
# Creat your Access Token to authenticate object
auth.set_access_token(ACCESS_TOKEN, ACCESS_TOKEN_SECRET)

In [10]:
# Create an API Object passing auth object
api = tw.API(auth)

In [11]:
# function to perform data extraction
def scrape(movieID, movie_title, words):
    try:
        numtweet = 100  

        # Creating DataFrame using pandas
        db = pd.DataFrame(columns=['Movie Id', 'Title', 'Tagword', 'description', 'totaltweets', 'retweetcount', 'text', 'hashtags'])

        # We are using .Cursor() to search through twitter for the required tweets.
        # The number of tweets can be restricted using .items(number of tweets)
        tweets = tw.Cursor(api.search_tweets, q=words, lang="en",
                               tweet_mode='extended').items(numtweet)

        # .Cursor() returns an iterable object. Each item in 
        # the iterator has various attributes that you can access to 
        # get information about each tweet
        list_tweets = [tweet for tweet in tweets]

        # Counter to maintain Tweet Count
        i = 1  

        # we will iterate over each tweet in the list for extracting information about each tweet
        for tweet in list_tweets:
            movie_id = movieID
            title = movie_title
            tagword = words
            description = tweet.user.description
            totaltweets = tweet.user.statuses_count
            retweetcount = tweet.retweet_count
            hashtags = tweet.entities['hashtags']

            # Retweets can be distinguished by a retweeted_status attribute,
            # in case it is an invalid reference, except block will be executed
            try:
                text = tweet.retweeted_status.full_text
            except AttributeError:
                text = tweet.full_text
            hashtext = list()
            for j in range(0, len(hashtags)):
                hashtext.append(hashtags[j]['text'])

            # Here we are appending all the extracted information in the DataFrame
            ith_tweet = [movie_id, title, tagword, description,
                         totaltweets, retweetcount, text, hashtext]
            db.loc[len(db)] = ith_tweet

            # Function call to print tweet data on screen
    #         print(ith_tweet)
            i = i+1
        filename = str(movie_title) + '_tweets.csv'

        # we will save our database as a CSV file.
        db.to_csv('tweets_collected/' +filename)
    
             
    except Exception as e:
        print(str(e))
        pass

In [12]:
for index, row in movieinfo[75:].iterrows():
    movieID = row['itemId']
    movie_title = row['title']
    movie_tagword = row['tagword']
    scrape(movieID, movie_title, movie_tagword)

429 Too Many Requests
88 - Rate limit exceeded
429 Too Many Requests
88 - Rate limit exceeded
429 Too Many Requests
88 - Rate limit exceeded
429 Too Many Requests
88 - Rate limit exceeded
429 Too Many Requests
88 - Rate limit exceeded
429 Too Many Requests
88 - Rate limit exceeded
429 Too Many Requests
88 - Rate limit exceeded
429 Too Many Requests
88 - Rate limit exceeded
429 Too Many Requests
88 - Rate limit exceeded
429 Too Many Requests
88 - Rate limit exceeded
429 Too Many Requests
88 - Rate limit exceeded
429 Too Many Requests
88 - Rate limit exceeded
429 Too Many Requests
88 - Rate limit exceeded
429 Too Many Requests
88 - Rate limit exceeded
429 Too Many Requests
88 - Rate limit exceeded
429 Too Many Requests
88 - Rate limit exceeded
429 Too Many Requests
88 - Rate limit exceeded
429 Too Many Requests
88 - Rate limit exceeded
429 Too Many Requests
88 - Rate limit exceeded
429 Too Many Requests
88 - Rate limit exceeded
429 Too Many Requests
88 - Rate limit exceeded
429 Too Many 

429 Too Many Requests
88 - Rate limit exceeded
429 Too Many Requests
88 - Rate limit exceeded
429 Too Many Requests
88 - Rate limit exceeded
429 Too Many Requests
88 - Rate limit exceeded
429 Too Many Requests
88 - Rate limit exceeded
429 Too Many Requests
88 - Rate limit exceeded
429 Too Many Requests
88 - Rate limit exceeded
429 Too Many Requests
88 - Rate limit exceeded
429 Too Many Requests
88 - Rate limit exceeded
429 Too Many Requests
88 - Rate limit exceeded
429 Too Many Requests
88 - Rate limit exceeded
429 Too Many Requests
88 - Rate limit exceeded
429 Too Many Requests
88 - Rate limit exceeded
429 Too Many Requests
88 - Rate limit exceeded
429 Too Many Requests
88 - Rate limit exceeded
429 Too Many Requests
88 - Rate limit exceeded
429 Too Many Requests
88 - Rate limit exceeded
429 Too Many Requests
88 - Rate limit exceeded
429 Too Many Requests
88 - Rate limit exceeded
429 Too Many Requests
88 - Rate limit exceeded
429 Too Many Requests
88 - Rate limit exceeded
429 Too Many 

429 Too Many Requests
88 - Rate limit exceeded
429 Too Many Requests
88 - Rate limit exceeded
429 Too Many Requests
88 - Rate limit exceeded
429 Too Many Requests
88 - Rate limit exceeded
429 Too Many Requests
88 - Rate limit exceeded
429 Too Many Requests
88 - Rate limit exceeded
429 Too Many Requests
88 - Rate limit exceeded
429 Too Many Requests
88 - Rate limit exceeded
429 Too Many Requests
88 - Rate limit exceeded
429 Too Many Requests
88 - Rate limit exceeded
429 Too Many Requests
88 - Rate limit exceeded
429 Too Many Requests
88 - Rate limit exceeded
429 Too Many Requests
88 - Rate limit exceeded
429 Too Many Requests
88 - Rate limit exceeded
429 Too Many Requests
88 - Rate limit exceeded
429 Too Many Requests
88 - Rate limit exceeded
429 Too Many Requests
88 - Rate limit exceeded
429 Too Many Requests
88 - Rate limit exceeded
429 Too Many Requests
88 - Rate limit exceeded
429 Too Many Requests
88 - Rate limit exceeded
429 Too Many Requests
88 - Rate limit exceeded
429 Too Many 

429 Too Many Requests
88 - Rate limit exceeded
429 Too Many Requests
88 - Rate limit exceeded
429 Too Many Requests
88 - Rate limit exceeded
429 Too Many Requests
88 - Rate limit exceeded
429 Too Many Requests
88 - Rate limit exceeded
429 Too Many Requests
88 - Rate limit exceeded
429 Too Many Requests
88 - Rate limit exceeded
429 Too Many Requests
88 - Rate limit exceeded
429 Too Many Requests
88 - Rate limit exceeded
429 Too Many Requests
88 - Rate limit exceeded
429 Too Many Requests
88 - Rate limit exceeded
429 Too Many Requests
88 - Rate limit exceeded
429 Too Many Requests
88 - Rate limit exceeded
429 Too Many Requests
88 - Rate limit exceeded
429 Too Many Requests
88 - Rate limit exceeded
429 Too Many Requests
88 - Rate limit exceeded
429 Too Many Requests
88 - Rate limit exceeded
429 Too Many Requests
88 - Rate limit exceeded
429 Too Many Requests
88 - Rate limit exceeded
429 Too Many Requests
88 - Rate limit exceeded
429 Too Many Requests
88 - Rate limit exceeded
429 Too Many 

429 Too Many Requests
88 - Rate limit exceeded
429 Too Many Requests
88 - Rate limit exceeded
429 Too Many Requests
88 - Rate limit exceeded
429 Too Many Requests
88 - Rate limit exceeded
429 Too Many Requests
88 - Rate limit exceeded
429 Too Many Requests
88 - Rate limit exceeded
429 Too Many Requests
88 - Rate limit exceeded
429 Too Many Requests
88 - Rate limit exceeded
429 Too Many Requests
88 - Rate limit exceeded
429 Too Many Requests
88 - Rate limit exceeded
429 Too Many Requests
88 - Rate limit exceeded
429 Too Many Requests
88 - Rate limit exceeded
429 Too Many Requests
88 - Rate limit exceeded
429 Too Many Requests
88 - Rate limit exceeded
429 Too Many Requests
88 - Rate limit exceeded
429 Too Many Requests
88 - Rate limit exceeded
429 Too Many Requests
88 - Rate limit exceeded
429 Too Many Requests
88 - Rate limit exceeded
429 Too Many Requests
88 - Rate limit exceeded
429 Too Many Requests
88 - Rate limit exceeded
429 Too Many Requests
88 - Rate limit exceeded
429 Too Many 

429 Too Many Requests
88 - Rate limit exceeded
429 Too Many Requests
88 - Rate limit exceeded
429 Too Many Requests
88 - Rate limit exceeded
429 Too Many Requests
88 - Rate limit exceeded
429 Too Many Requests
88 - Rate limit exceeded
429 Too Many Requests
88 - Rate limit exceeded
429 Too Many Requests
88 - Rate limit exceeded
429 Too Many Requests
88 - Rate limit exceeded
429 Too Many Requests
88 - Rate limit exceeded
429 Too Many Requests
88 - Rate limit exceeded
429 Too Many Requests
88 - Rate limit exceeded
429 Too Many Requests
88 - Rate limit exceeded
429 Too Many Requests
88 - Rate limit exceeded
429 Too Many Requests
88 - Rate limit exceeded
429 Too Many Requests
88 - Rate limit exceeded
429 Too Many Requests
88 - Rate limit exceeded
429 Too Many Requests
88 - Rate limit exceeded
429 Too Many Requests
88 - Rate limit exceeded
429 Too Many Requests
88 - Rate limit exceeded
429 Too Many Requests
88 - Rate limit exceeded
429 Too Many Requests
88 - Rate limit exceeded
429 Too Many 

429 Too Many Requests
88 - Rate limit exceeded
429 Too Many Requests
88 - Rate limit exceeded
429 Too Many Requests
88 - Rate limit exceeded
429 Too Many Requests
88 - Rate limit exceeded
429 Too Many Requests
88 - Rate limit exceeded
429 Too Many Requests
88 - Rate limit exceeded
429 Too Many Requests
88 - Rate limit exceeded
429 Too Many Requests
88 - Rate limit exceeded
429 Too Many Requests
88 - Rate limit exceeded
429 Too Many Requests
88 - Rate limit exceeded
429 Too Many Requests
88 - Rate limit exceeded
429 Too Many Requests
88 - Rate limit exceeded
429 Too Many Requests
88 - Rate limit exceeded
429 Too Many Requests
88 - Rate limit exceeded
429 Too Many Requests
88 - Rate limit exceeded
429 Too Many Requests
88 - Rate limit exceeded
429 Too Many Requests
88 - Rate limit exceeded
429 Too Many Requests
88 - Rate limit exceeded
429 Too Many Requests
88 - Rate limit exceeded
429 Too Many Requests
88 - Rate limit exceeded
429 Too Many Requests
88 - Rate limit exceeded
429 Too Many 

429 Too Many Requests
88 - Rate limit exceeded
429 Too Many Requests
88 - Rate limit exceeded
429 Too Many Requests
88 - Rate limit exceeded
429 Too Many Requests
88 - Rate limit exceeded
429 Too Many Requests
88 - Rate limit exceeded
429 Too Many Requests
88 - Rate limit exceeded
429 Too Many Requests
88 - Rate limit exceeded
429 Too Many Requests
88 - Rate limit exceeded
429 Too Many Requests
88 - Rate limit exceeded
429 Too Many Requests
88 - Rate limit exceeded
429 Too Many Requests
88 - Rate limit exceeded
429 Too Many Requests
88 - Rate limit exceeded
429 Too Many Requests
88 - Rate limit exceeded
429 Too Many Requests
88 - Rate limit exceeded
429 Too Many Requests
88 - Rate limit exceeded
429 Too Many Requests
88 - Rate limit exceeded
429 Too Many Requests
88 - Rate limit exceeded
429 Too Many Requests
88 - Rate limit exceeded
429 Too Many Requests
88 - Rate limit exceeded
429 Too Many Requests
88 - Rate limit exceeded
429 Too Many Requests
88 - Rate limit exceeded
429 Too Many 

429 Too Many Requests
88 - Rate limit exceeded
429 Too Many Requests
88 - Rate limit exceeded
429 Too Many Requests
88 - Rate limit exceeded
429 Too Many Requests
88 - Rate limit exceeded
429 Too Many Requests
88 - Rate limit exceeded
429 Too Many Requests
88 - Rate limit exceeded
429 Too Many Requests
88 - Rate limit exceeded
429 Too Many Requests
88 - Rate limit exceeded
429 Too Many Requests
88 - Rate limit exceeded
429 Too Many Requests
88 - Rate limit exceeded
429 Too Many Requests
88 - Rate limit exceeded
429 Too Many Requests
88 - Rate limit exceeded
429 Too Many Requests
88 - Rate limit exceeded
429 Too Many Requests
88 - Rate limit exceeded
429 Too Many Requests
88 - Rate limit exceeded
429 Too Many Requests
88 - Rate limit exceeded
429 Too Many Requests
88 - Rate limit exceeded
429 Too Many Requests
88 - Rate limit exceeded
429 Too Many Requests
88 - Rate limit exceeded
429 Too Many Requests
88 - Rate limit exceeded
429 Too Many Requests
88 - Rate limit exceeded
429 Too Many 

In [13]:
# specifying the path to csv files
path = "tweets_collected"
  
# csv files in the path
files = glob.glob(path + "/*.csv")
  
print(len(files))
  
# defining an empty list to store 
# content
data_frame = pd.DataFrame()
content = []
  
# checking all the csv files in the 
# specified path
for filename in files:
    
    # reading content of csv file
#     content.append(filename)
    df = pd.read_csv(filename, index_col=None)
    content.append(df)

    
# converting content to data frame
data_frame = pd.concat(content)
data_frame.drop(['Unnamed: 0'], axis=1, inplace=True)
data_frame

45


,Movie Id,Title,Tagword,description,totaltweets,retweetcount,text,hashtags
0,95,Aladdin,#aladdin,Disney lover #elenaofavalor\n#sofiathefirst\nP...,22852,7,Today marks the 27th anniversary of the #Aladd...,['Aladdin']
1,95,Aladdin,#aladdin,North Wales premier multi purpose venue. Capac...,4484,1,Less than 24 hours until #aladdin #pantomine o...,"['aladdin', 'pantomine']"
2,95,Aladdin,#aladdin,DM links for 2v2 chall. COD Trickshotz origina...,125613,1,#Aladdin is now streaming #Call of Duty: Warzo...,"['Aladdin', 'Call']"
3,95,Aladdin,#aladdin,"i like anime, wwe, and video games",40547,1,#Aladdin is now streaming #Call of Duty: Warzo...,"['Aladdin', 'Call']"
4,95,Aladdin,#aladdin,Twitter account for the fan blog for Disney's ...,20389,0,So thankful for Michael Maliakel &amp; @shoba_...,['Aladdin']
...,...,...,...,...,...,...,...,...
95,118,Twister,#twister,TooTwo is an exclusive firm built for kids off...,373,0,Let them ride freely with the big panda twiste...,"['tootwo', 'tootwotoys', 'tootwoonline', 'kids..."
96,118,Twister,#twister,Preschool teacher. \nFights bad guys with shar...,67915,0,@Elpopularis @blu_kryptonian @fizzily1 @mopeng...,['Twister']
97,118,Twister,#twister,The Best #StormChasing tours in Tornado Alley!...,137683,0,Is #Twister on #Netflix? It should be! #stormc...,"['Twister', 'Netflix', 'stormchasing']"
98,118,Twister,#twister,"Mum of 5, former mua, 2 fur babies 2nd account",783,0,"Right hand ✋ red, right foot 🦶 green #twister ...",['twister']


In [14]:
# Create object of stopwords
stopwords = nltk.corpus.stopwords.words('english')

# Cleaning Tweets

In [15]:
def clean_text(text):
    text = "".join([word for word in text if word not in string.punctuation])
    tokens = re.split('\W+', text)
    text = " ".join([word for word in tokens if word not in stopwords])
    return text

data_frame['Cleaned_tweets'] = data_frame['text'].apply(lambda x: clean_text(x.lower()))
data_frame

,Movie Id,Title,Tagword,description,totaltweets,retweetcount,text,hashtags,Cleaned_tweets
0,95,Aladdin,#aladdin,Disney lover #elenaofavalor\n#sofiathefirst\nP...,22852,7,Today marks the 27th anniversary of the #Aladd...,['Aladdin'],today marks 27th anniversary aladdin episode e...
1,95,Aladdin,#aladdin,North Wales premier multi purpose venue. Capac...,4484,1,Less than 24 hours until #aladdin #pantomine o...,"['aladdin', 'pantomine']",less 24 hours aladdin pantomine opens venuecym...
2,95,Aladdin,#aladdin,DM links for 2v2 chall. COD Trickshotz origina...,125613,1,#Aladdin is now streaming #Call of Duty: Warzo...,"['Aladdin', 'Call']",aladdin streaming call duty warzone httpstcokl...
3,95,Aladdin,#aladdin,"i like anime, wwe, and video games",40547,1,#Aladdin is now streaming #Call of Duty: Warzo...,"['Aladdin', 'Call']",aladdin streaming call duty warzone httpstcokl...
4,95,Aladdin,#aladdin,Twitter account for the fan blog for Disney's ...,20389,0,So thankful for Michael Maliakel &amp; @shoba_...,['Aladdin'],thankful michael maliakel amp shobanarayan per...
...,...,...,...,...,...,...,...,...,...
95,118,Twister,#twister,TooTwo is an exclusive firm built for kids off...,373,0,Let them ride freely with the big panda twiste...,"['tootwo', 'tootwotoys', 'tootwoonline', 'kids...",let ride freely big panda twister magic car bu...
96,118,Twister,#twister,Preschool teacher. \nFights bad guys with shar...,67915,0,@Elpopularis @blu_kryptonian @fizzily1 @mopeng...,['Twister'],elpopularis blukryptonian fizzily1 mopeng tent...
97,118,Twister,#twister,The Best #StormChasing tours in Tornado Alley!...,137683,0,Is #Twister on #Netflix? It should be! #stormc...,"['Twister', 'Netflix', 'stormchasing']",twister netflix stormchasing
98,118,Twister,#twister,"Mum of 5, former mua, 2 fur babies 2nd account",783,0,"Right hand ✋ red, right foot 🦶 green #twister ...",['twister'],right hand red right foot green twister tummy ...


# Vader Sentiment Analysis

In [16]:
positive_sentiments=[]
negative_sentiments=[]
neutral_sentiments=[]
label_sentiments=[]

def sentiment_scores(comment):
 
    sid_obj = SentimentIntensityAnalyzer()
 
    sentiment_dict = sid_obj.polarity_scores(comment)
     
    print("Overall sentiment dictionary is : ", sentiment_dict)
    print("Movie was rated as ",negative_sentiments.append(sentiment_dict['neg']*100), "% Negative")
    print("Movie was rated as ",neutral_sentiments.append(sentiment_dict['neu']*100), "% Neutral")
    print("Movie was rated as ",positive_sentiments.append(sentiment_dict['pos']*100), "% Positive")
 
    print("Sentence Overall Rated As", end = " ")
 
    if sentiment_dict['compound'] > 0 :
        label_sentiments.append("Positive")
 
    elif sentiment_dict['compound'] < 0:
        label_sentiments.append("Negative")
        
    elif sentiment_dict['compound'] == 0:
        label_sentiments.append("Neutral")

In [20]:
if __name__ == "__main__" :
 
    data_frame['Cleaned_tweets'].apply(lambda x: sentiment_scores(x))

Overall sentiment dictionary is :  {'neg': 0.0, 'neu': 0.84, 'pos': 0.16, 'compound': 0.6369}
Movie was rated as  None % Negative
Movie was rated as  None % Neutral
Movie was rated as  None % Positive
Sentence Overall Rated As Overall sentiment dictionary is :  {'neg': 0.0, 'neu': 0.789, 'pos': 0.211, 'compound': 0.4939}
Movie was rated as  None % Negative
Movie was rated as  None % Neutral
Movie was rated as  None % Positive
Sentence Overall Rated As Overall sentiment dictionary is :  {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
Movie was rated as  None % Negative
Movie was rated as  None % Neutral
Movie was rated as  None % Positive
Sentence Overall Rated As Overall sentiment dictionary is :  {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
Movie was rated as  None % Negative
Movie was rated as  None % Neutral
Movie was rated as  None % Positive
Sentence Overall Rated As Overall sentiment dictionary is :  {'neg': 0.0, 'neu': 0.51, 'pos': 0.49, 'compound': 0.9686}
Movie 

Sentence Overall Rated As Overall sentiment dictionary is :  {'neg': 0.0, 'neu': 0.62, 'pos': 0.38, 'compound': 0.9022}
Movie was rated as  None % Negative
Movie was rated as  None % Neutral
Movie was rated as  None % Positive
Sentence Overall Rated As Overall sentiment dictionary is :  {'neg': 0.0, 'neu': 0.577, 'pos': 0.423, 'compound': 0.9601}
Movie was rated as  None % Negative
Movie was rated as  None % Neutral
Movie was rated as  None % Positive
Sentence Overall Rated As Overall sentiment dictionary is :  {'neg': 0.0, 'neu': 0.511, 'pos': 0.489, 'compound': 0.7717}
Movie was rated as  None % Negative
Movie was rated as  None % Neutral
Movie was rated as  None % Positive
Sentence Overall Rated As Overall sentiment dictionary is :  {'neg': 0.0, 'neu': 0.85, 'pos': 0.15, 'compound': 0.4588}
Movie was rated as  None % Negative
Movie was rated as  None % Neutral
Movie was rated as  None % Positive
Sentence Overall Rated As Overall sentiment dictionary is :  {'neg': 0.0, 'neu': 1.0, 'p

Sentence Overall Rated As Overall sentiment dictionary is :  {'neg': 0.0, 'neu': 0.774, 'pos': 0.226, 'compound': 0.5859}
Movie was rated as  None % Negative
Movie was rated as  None % Neutral
Movie was rated as  None % Positive
Sentence Overall Rated As Overall sentiment dictionary is :  {'neg': 0.0, 'neu': 0.75, 'pos': 0.25, 'compound': 0.4588}
Movie was rated as  None % Negative
Movie was rated as  None % Neutral
Movie was rated as  None % Positive
Sentence Overall Rated As Overall sentiment dictionary is :  {'neg': 0.0, 'neu': 0.75, 'pos': 0.25, 'compound': 0.4588}
Movie was rated as  None % Negative
Movie was rated as  None % Neutral
Movie was rated as  None % Positive
Sentence Overall Rated As Overall sentiment dictionary is :  {'neg': 0.0, 'neu': 0.747, 'pos': 0.253, 'compound': 0.8316}
Movie was rated as  None % Negative
Movie was rated as  None % Neutral
Movie was rated as  None % Positive
Sentence Overall Rated As Overall sentiment dictionary is :  {'neg': 0.0, 'neu': 1.0, 'p

Sentence Overall Rated As Overall sentiment dictionary is :  {'neg': 0.0, 'neu': 0.809, 'pos': 0.191, 'compound': 0.7096}
Movie was rated as  None % Negative
Movie was rated as  None % Neutral
Movie was rated as  None % Positive
Sentence Overall Rated As Overall sentiment dictionary is :  {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
Movie was rated as  None % Negative
Movie was rated as  None % Neutral
Movie was rated as  None % Positive
Sentence Overall Rated As Overall sentiment dictionary is :  {'neg': 0.0, 'neu': 0.698, 'pos': 0.302, 'compound': 0.5994}
Movie was rated as  None % Negative
Movie was rated as  None % Neutral
Movie was rated as  None % Positive
Sentence Overall Rated As Overall sentiment dictionary is :  {'neg': 0.086, 'neu': 0.661, 'pos': 0.253, 'compound': 0.7003}
Movie was rated as  None % Negative
Movie was rated as  None % Neutral
Movie was rated as  None % Positive
Sentence Overall Rated As Overall sentiment dictionary is :  {'neg': 0.0, 'neu': 1.0, 'po

Sentence Overall Rated As Overall sentiment dictionary is :  {'neg': 0.0, 'neu': 0.663, 'pos': 0.337, 'compound': 0.6808}
Movie was rated as  None % Negative
Movie was rated as  None % Neutral
Movie was rated as  None % Positive
Sentence Overall Rated As Overall sentiment dictionary is :  {'neg': 0.0, 'neu': 0.892, 'pos': 0.108, 'compound': 0.5423}
Movie was rated as  None % Negative
Movie was rated as  None % Neutral
Movie was rated as  None % Positive
Sentence Overall Rated As Overall sentiment dictionary is :  {'neg': 0.049, 'neu': 0.951, 'pos': 0.0, 'compound': -0.1027}
Movie was rated as  None % Negative
Movie was rated as  None % Neutral
Movie was rated as  None % Positive
Sentence Overall Rated As Overall sentiment dictionary is :  {'neg': 0.0, 'neu': 0.556, 'pos': 0.444, 'compound': 0.9081}
Movie was rated as  None % Negative
Movie was rated as  None % Neutral
Movie was rated as  None % Positive
Sentence Overall Rated As Overall sentiment dictionary is :  {'neg': 0.0, 'neu': 1.

...

Sentence Overall Rated As Overall sentiment dictionary is :  {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
Movie was rated as  None % Negative
Movie was rated as  None % Neutral
Movie was rated as  None % Positive
Sentence Overall Rated As Overall sentiment dictionary is :  {'neg': 0.0, 'neu': 0.604, 'pos': 0.396, 'compound': 0.7096}
Movie was rated as  None % Negative
Movie was rated as  None % Neutral
Movie was rated as  None % Positive
Sentence Overall Rated As Overall sentiment dictionary is :  {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
Movie was rated as  None % Negative
Movie was rated as  None % Neutral
Movie was rated as  None % Positive
Sentence Overall Rated As Overall sentiment dictionary is :  {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
Movie was rated as  None % Negative
Movie was rated as  None % Neutral
Movie was rated as  None % Positive
Sentence Overall Rated As Overall sentiment dictionary is :  {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compou

Sentence Overall Rated As Overall sentiment dictionary is :  {'neg': 0.286, 'neu': 0.714, 'pos': 0.0, 'compound': -0.1531}
Movie was rated as  None % Negative
Movie was rated as  None % Neutral
Movie was rated as  None % Positive
Sentence Overall Rated As Overall sentiment dictionary is :  {'neg': 0.315, 'neu': 0.685, 'pos': 0.0, 'compound': -0.3182}
Movie was rated as  None % Negative
Movie was rated as  None % Neutral
Movie was rated as  None % Positive
Sentence Overall Rated As Overall sentiment dictionary is :  {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
Movie was rated as  None % Negative
Movie was rated as  None % Neutral
Movie was rated as  None % Positive
Sentence Overall Rated As Overall sentiment dictionary is :  {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
Movie was rated as  None % Negative
Movie was rated as  None % Neutral
Movie was rated as  None % Positive
Sentence Overall Rated As Overall sentiment dictionary is :  {'neg': 0.067, 'neu': 0.514, 'pos':

Sentence Overall Rated As Overall sentiment dictionary is :  {'neg': 0.176, 'neu': 0.495, 'pos': 0.33, 'compound': 0.4215}
Movie was rated as  None % Negative
Movie was rated as  None % Neutral
Movie was rated as  None % Positive
Sentence Overall Rated As Overall sentiment dictionary is :  {'neg': 0.277, 'neu': 0.723, 'pos': 0.0, 'compound': -0.3182}
Movie was rated as  None % Negative
Movie was rated as  None % Neutral
Movie was rated as  None % Positive
Sentence Overall Rated As Overall sentiment dictionary is :  {'neg': 0.0, 'neu': 0.508, 'pos': 0.492, 'compound': 0.4404}
Movie was rated as  None % Negative
Movie was rated as  None % Neutral
Movie was rated as  None % Positive
Sentence Overall Rated As Overall sentiment dictionary is :  {'neg': 0.149, 'neu': 0.621, 'pos': 0.23, 'compound': 0.3182}
Movie was rated as  None % Negative
Movie was rated as  None % Neutral
Movie was rated as  None % Positive
Sentence Overall Rated As Overall sentiment dictionary is :  {'neg': 0.0, 'neu': 

Sentence Overall Rated As Overall sentiment dictionary is :  {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
Movie was rated as  None % Negative
Movie was rated as  None % Neutral
Movie was rated as  None % Positive
Sentence Overall Rated As Overall sentiment dictionary is :  {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
Movie was rated as  None % Negative
Movie was rated as  None % Neutral
Movie was rated as  None % Positive
Sentence Overall Rated As Overall sentiment dictionary is :  {'neg': 0.0, 'neu': 0.906, 'pos': 0.094, 'compound': 0.4588}
Movie was rated as  None % Negative
Movie was rated as  None % Neutral
Movie was rated as  None % Positive
Sentence Overall Rated As Overall sentiment dictionary is :  {'neg': 0.181, 'neu': 0.603, 'pos': 0.216, 'compound': 0.1027}
Movie was rated as  None % Negative
Movie was rated as  None % Neutral
Movie was rated as  None % Positive
Sentence Overall Rated As Overall sentiment dictionary is :  {'neg': 0.0, 'neu': 0.732, 'pos': 0

Sentence Overall Rated As Overall sentiment dictionary is :  {'neg': 0.135, 'neu': 0.865, 'pos': 0.0, 'compound': -0.3612}
Movie was rated as  None % Negative
Movie was rated as  None % Neutral
Movie was rated as  None % Positive
Sentence Overall Rated As Overall sentiment dictionary is :  {'neg': 0.12, 'neu': 0.88, 'pos': 0.0, 'compound': -0.128}
Movie was rated as  None % Negative
Movie was rated as  None % Neutral
Movie was rated as  None % Positive
Sentence Overall Rated As Overall sentiment dictionary is :  {'neg': 0.0, 'neu': 0.873, 'pos': 0.127, 'compound': 0.4404}
Movie was rated as  None % Negative
Movie was rated as  None % Neutral
Movie was rated as  None % Positive
Sentence Overall Rated As Overall sentiment dictionary is :  {'neg': 0.094, 'neu': 0.906, 'pos': 0.0, 'compound': -0.4215}
Movie was rated as  None % Negative
Movie was rated as  None % Neutral
Movie was rated as  None % Positive
Sentence Overall Rated As Overall sentiment dictionary is :  {'neg': 0.0, 'neu': 1.0

In [21]:
data_frame['Positive_Score']=positive_sentiments
data_frame['Negative_Score']=negative_sentiments
data_frame['Neutral_Score']=neutral_sentiments
data_frame['Sentiments_Category']=label_sentiments

In [25]:
data_frame

,Movie Id,Title,Tagword,description,totaltweets,retweetcount,text,hashtags,Cleaned_tweets,Positive_Score,Negative_Score,Neutral_Score,Sentiments_Category
0,95,Aladdin,#aladdin,Disney lover #elenaofavalor\n#sofiathefirst\nP...,22852,7,Today marks the 27th anniversary of the #Aladd...,['Aladdin'],today marks 27th anniversary aladdin episode e...,16.0,0.0,84.0,Positive
1,95,Aladdin,#aladdin,North Wales premier multi purpose venue. Capac...,4484,1,Less than 24 hours until #aladdin #pantomine o...,"['aladdin', 'pantomine']",less 24 hours aladdin pantomine opens venuecym...,21.1,0.0,78.9,Positive
2,95,Aladdin,#aladdin,DM links for 2v2 chall. COD Trickshotz origina...,125613,1,#Aladdin is now streaming #Call of Duty: Warzo...,"['Aladdin', 'Call']",aladdin streaming call duty warzone httpstcokl...,0.0,0.0,100.0,Neutral
3,95,Aladdin,#aladdin,"i like anime, wwe, and video games",40547,1,#Aladdin is now streaming #Call of Duty: Warzo...,"['Aladdin', 'Call']",aladdin streaming call duty warzone httpstcokl...,0.0,0.0,100.0,Neutral
4,95,Aladdin,#aladdin,Twitter account for the fan blog for Disney's ...,20389,0,So thankful for Michael Maliakel &amp; @shoba_...,['Aladdin'],thankful michael maliakel amp shobanarayan per...,49.0,0.0,51.0,Positive
...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,118,Twister,#twister,TooTwo is an exclusive firm built for kids off...,373,0,Let them ride freely with the big panda twiste...,"['tootwo', 'tootwotoys', 'tootwoonline', 'kids...",let ride freely big panda twister magic car bu...,12.7,0.0,87.3,Positive
96,118,Twister,#twister,Preschool teacher. \nFights bad guys with shar...,67915,0,@Elpopularis @blu_kryptonian @fizzily1 @mopeng...,['Twister'],elpopularis blukryptonian fizzily1 mopeng tent...,0.0,9.4,90.6,Negative
97,118,Twister,#twister,The Best #StormChasing tours in Tornado Alley!...,137683,0,Is #Twister on #Netflix? It should be! #stormc...,"['Twister', 'Netflix', 'stormchasing']",twister netflix stormchasing,0.0,0.0,100.0,Neutral
98,118,Twister,#twister,"Mum of 5, former mua, 2 fur babies 2nd account",783,0,"Right hand ✋ red, right foot 🦶 green #twister ...",['twister'],right hand red right foot green twister tummy ...,24.2,0.0,75.8,Positive


In [26]:
data_frame.to_csv('VSA_on_tweets.csv')